# Tutorial: Variables (One-Shot `evaluate(...)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/variable.html

This is the one-shot variant: each line is an independent `evaluate("...")` call.


In [ ]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")


## Before You Start

1. Run the setup cell first (`from bqn_jax import evaluate`).
2. Then run cells from top to bottom.
3. Each `evaluate("...")` call parses and runs only that snippet.
4. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## Examples

Quick warm-up examples before the deeper sections.


In [ ]:
evaluate('hey ← "Hi there"')
evaluate('hey ∾ ", World!"')


## Defining variables

Define names with `←`, then update existing names with `↩` when needed.


In [ ]:
evaluate('pi‿e‿ten ← ⟨ π, ⋆1, 10 ⟩')
evaluate('ten × pi')
evaluate('three ⋈ ten - three ← 3')


In [ ]:
evaluate('three ← 4')


In [ ]:
evaluate('three ↩ 4')
evaluate('three = 3   # Wait why did I do that')
evaluate('3 = three ↩ 3')
evaluate("four ↩ 3    # four isn't defined yet")


## Variable roles

Identifier spelling and casing affect syntactic role in BQN.


In [ ]:
evaluate('BQN ← "[many pages of specification]"')


In [ ]:
evaluate('three')
evaluate('thrEe')
evaluate('ThReE')
evaluate('thr_EE')
evaluate('__three')
evaluate('_T_H_R_E_E_')


In [ ]:
evaluate('- Three')
evaluate('- _three')


In [ ]:
evaluate('1_000_000')


## Function assignment

Functions are first-class values, so assignment works for function definitions too.


In [ ]:
evaluate('Base2 ← +⟜(2⊸×)´∘⌽')
evaluate('Base2 1‿0‿1‿0')
evaluate('Base2 "01010001"-\'0\'')
evaluate('@ + Base2¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


In [ ]:
evaluate('Base2')
evaluate('base2 ↩ 16   # Change it to a number')
evaluate('Base2')
evaluate('Base2 6')


## Modifying part of an array

These examples show targeted updates through functional transforms like Under.


In [ ]:
evaluate('"BQN"            # A list of characters')
evaluate('-⟜1⌾(2⊸⊑) "BQN"  # Wait why did I do that')


In [ ]:
evaluate('(↕3) ⋈¨ "BQN"')
evaluate('1 ⊑ "BQN"')


In [ ]:
evaluate('8⌾⊑ "BQN"        # Change the first element to 8')


In [ ]:
evaluate('↕7')
evaluate('4 ↑ ↕7           # The first four elements')
evaluate('⌽⌾(4⊸↑) ↕7       # And reverse them')
evaluate('⌽⌾(¯4⊸↑) ↕7      # Or reverse the last four')


In [ ]:
evaluate('2⊸⌽⌾(1⊸⊑) "xyz"‿"ABCDE"‿"wxyz"‿"yz"')
evaluate('2⊸⌽⌾(2⊸↓) "XYabcde"')


In [ ]:
evaluate('¯3 ↓ "abcdefgh"')
evaluate('2 ↑ 4 ↓ "abcdefgh"')


In [ ]:
evaluate('(\'A\'-\'a\')⊸+ ⌾ (2 ↑ 4⊸↓)  "abcdefgh"')


## Identity functions

Identity-like forms are useful when controlling data flow in larger expressions.


In [ ]:
evaluate('⊢ "only"')
evaluate('⊣ "only"')
evaluate('"left" ⊢ "right"')
evaluate('"left" ⊣ "right"')


## Modified assignment

Combine assignment with transforms to update values concisely.


In [ ]:
evaluate("a ← 4            # First it's a number\na")
evaluate("a ↩ 4‿5‿6        # Now it's a list!\na")


In [ ]:
evaluate('a ↩ a - 1\na')
evaluate('a -↩ 1')


In [ ]:
evaluate('a ∾˜↩ 0‿1')


In [ ]:
evaluate('"abcd" ⌽∘⊣ "wxyz"')
evaluate('a ⌽∘⊣↩ @')


In [ ]:
evaluate('a ⌽↩')
evaluate('a 4⊸-↩           # And back again')


In [ ]:
evaluate('-⟜4⌾(¯2⊸↑) a')
evaluate("a                # It hasn't changed, of course")


In [ ]:
evaluate('a -⟜4⌾(¯2⊸↑)↩')
